In [16]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\as041\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\as041\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\as041\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
from keras.layers import Dense, Dropout
from keras.models import Sequential


In [18]:
import numpy
import random
import json


In [19]:
intents = {}
with open("intents.json") as f:
    intents = json.load(f)
print("Intents Loaded as Dataset")


Intents Loaded as Dataset


In [20]:
words = []
classes = []
documents = []


In [21]:
for intent in intents['intents']:
    classes.append(intent['tag'])
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))


In [22]:
ignore_words = ['?', '!', ',']
words = [lemmatizer.lemmatize(w.lower())
         for w in words if w not in ignore_words]


In [23]:
print(len(documents), "Documents")

classes = sorted(list(set(classes)))
print(len(classes), "Classes")

words = sorted(list(set(words)))
print(len(words), "Unique Lemmatized Words")


329 Documents
71 Classes
147 Unique Lemmatized Words


In [24]:
with open("words.json", "w") as f:
    json.dump(words, f)
    print("Words Done")

with open("classes.json", "w") as f:
    json.dump(classes, f)
    print("Classes Done")

with open("documents.json", "w") as f:
    json.dump(documents, f)
    print("Documents Done")


Words Done
Classes Done
Dosuments Done


In [25]:
training = []

for doc in documents:
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(
        word.lower()) for word in pattern_words]

    bag = []
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list([0] * len(classes))
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
    
random.shuffle(training)
training = numpy.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training Data Created")


Training Data Created


In [26]:
model = Sequential()

model.add(Dense(256, input_shape=(
    len(train_x[0]),), activation='relu', name="layer_1"))
model.add(Dropout(0.5, name="dropout_1"))
model.add(Dense(128, activation='relu', name="layer_2"))
model.add(Dropout(0.5, name="dropout_2"))
model.add(Dense(len(train_y[0]), activation='softmax', name="layer_3"))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='RMSProp', metrics=['accuracy'])

Saving_Model = model.fit(numpy.array(train_x), numpy.array(train_y),
          epochs=200, batch_size=20)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 256)               37888     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 layer_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 layer_3 (Dense)             (None, 71)                9159      
                                                                 
Total params: 79,943
Trainable params: 79,943
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
17/17 [==============================] -

In [27]:
score = model.evaluate(numpy.array(
    train_x), numpy.array(train_y), verbose=True)

print("Model Loss: ", score[0])

print("Model Accuracy: ", score[1])


11/11 [==============================] - 0s 2ms/step - loss: 1.0013e-04 - accuracy: 1.0000
Model Loss:  0.00010012574057327583
Model Accuracy:  1.0


In [28]:
model.save("Chatbot_Model.h5", Saving_Model)
print("Model Created")

Model Created
